# Load in Data

In [159]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [160]:
estancias = gpd.read_file("Estancias_corrals_from_sat_imagery")
estancias_crs = estancias.crs
data = gpd.read_file('wernke_logres.csv')
name_mapper =  {"Dataset":"label",
               "T0":"solar_radiance",
               "TRI1":"terrain_roughness",
               "TPI1":"terrain_prominence",
               "Slope1":"slope",
               "Elevation1":"elevation",
               "AvgTemp1":"average_temperature",
               "Precip1":"annual_precipitation",
               #"fid_1":"fid",
               "LULC1":"land_use_class"}
data = data.rename(columns=name_mapper)   
keep_cols = [col for col in name_mapper.values()] + ['lat','lon']
data = data[keep_cols].copy()
data.replace('','0',inplace=True)
data = data.astype(float)
data['label'] = data['label'].astype(bool)
data_high =  data[data['elevation'] > 3000].copy()

# Check Point Distributions

In [161]:
positives = data_high[data_high['label']]
negatives = data_high[~data_high['label']]

In [158]:
feat = "winter_ndvi"
plt.hist(positives[feat], alpha = 0.5,density = True, label = "Estancias", bins = 30)
plt.hist(negatives[feat], alpha = 0.5,density = True, label = "Survey Area", bins = 30)
plt.title(f"Density of {feat} Estancia vs Survey Area")
plt.xlabel(feat)
plt.legend()

KeyError: 'winter_ndvi'

In [153]:
data['land_use_class'].unique()

array([130., 150., 120., 110., 200.,  50., 100., 153.,  11.,  10., 210.,
        30., 180.,  40., 220., 190.,  61.,  60.])

# Add NDVI data

In [162]:
import hs_geofuncs as hsg

In [163]:
years = ["2018","2019","2020"]
months = ["03","09"]
for year in years:
    for month in months:
        print("Evaluating vegetation indices in the time range "+year+"-"+month)
        data = hsg.add_data(data, 
                       ["ndvi-"+year+"-"+month,"evi-"+year+"-"+month],
                       collection = "modis-13Q1-061",
                       bands = ["250m_16_days_NDVI","250m_16_days_EVI"],
                       time_range = year+"-"+month,
                       crs = estancias_crs,
                       odc_args = {"crs":"EPSG:3857",
                                "resolution":250},
                       shapely_in_bds = True,
                       verbose = 1)

Evaluating vegetation indices in the time range 2018-03


APIError: <!DOCTYPE html PUBLIC '-//W3C//DTD XHTML 1.0 Transitional//EN' 'http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd'><html xmlns='http://www.w3.org/1999/xhtml'><head><meta content='text/html; charset=utf-8' http-equiv='content-type'/><style type='text/css'>body {font-family:Arial; margin-left:40px; }img  { border:0 none; }#content { margin-left: auto; margin-right: auto }#message h2 { font-size: 20px; font-weight: normal; color: #000000; margin: 34px 0px 0px 0px }#message p  { font-size: 13px; color: #000000; margin: 7px 0px 0px0px}#errorref { font-size: 11px; color: #737373; margin-top: 41px }</style><title>Service unavailable</title></head><body><div id='content'><div id='message'><h2>Our services aren't available right now</h2><p>We're working to restore all services as soon as possible. Please check back soon.</p></div><div id='errorref'><span>09KNBZAAAAABpMdUwy+moTJtrIxzOYBkjQlJVMzBFREdFMTEwNwA5MjdhYmZhNi0xOWY2LTRhZjEtYTA5ZC1jOTU5ZDlhMWU2NDQ=</span></div></div></body

In [108]:
data['summer_ndvi'] = (data['ndvi-2018-03'] + data['ndvi-2019-03']+data['ndvi-2020-03']) / 30_000
data['winter_ndvi'] = (data['ndvi-2018-09'] + data['ndvi-2019-09']+data['ndvi-2020-09']) / 30_000
data['elevation_from_mean'] = np.abs(data['elevation']-data[data['label']]['elevation'].mean())
data['slope_from_mean'] = np.abs(data['slope']-data[data['label']]['slope'].mean())

In [109]:
data['label'] = data['label'].astype(bool)

# Logistic Regression

In [113]:
data_high =  data[data['elevation'] > 3000].copy()

In [124]:
data

,label,solar_radiance,terrain_roughness,terrain_prominence,slope,elevation,average_temperature,annual_precipitation,land_use_class,lat,...,ndvi-2019-09,evi-2019-09,ndvi-2020-03,evi-2020-03,ndvi-2020-09,evi-2020-09,summer_ndvi,winter_ndvi,elevation_from_mean,slope_from_mean
0,True,2.316422e+06,25.000,-5.000,5.572638,3879.0,58.0,390.0,130.0,-15.505033,...,2241.0,1481.0,6044.0,3914.0,2331.0,1596.0,0.595167,0.219700,458.756907,4.344417
1,True,2.370194e+06,34.750,-5.750,4.797690,3864.0,62.0,285.0,130.0,-15.530761,...,2715.0,1871.0,5646.0,3446.0,2579.0,1760.0,0.630733,0.261833,473.756907,5.119364
2,True,2.258059e+06,50.250,11.250,7.122169,3739.0,68.0,286.0,130.0,-15.607778,...,2064.0,1206.0,4141.0,2015.0,2254.0,1265.0,0.428167,0.203400,598.756907,2.794886
3,True,2.506773e+06,18.625,4.000,7.312814,4513.0,31.0,247.0,150.0,-16.910971,...,1153.0,754.0,1557.0,843.0,1248.0,624.0,0.149900,0.117800,175.243093,2.604241
4,True,2.502171e+06,50.500,7.125,12.713185,4514.0,36.0,379.0,150.0,-17.040634,...,1480.0,936.0,2287.0,1024.0,1642.0,1024.0,0.210500,0.154133,176.243093,2.796131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19033,False,2.487665e+06,3.625,2.000,1.699721,4523.0,33.0,268.0,120.0,-16.335651,...,281.0,170.0,386.0,219.0,362.0,210.0,0.037833,0.029733,185.243093,8.217333
19034,False,2.504403e+06,5.625,-1.375,1.030675,4513.0,36.0,725.0,130.0,-15.507890,...,2534.0,1596.0,5354.0,3411.0,4637.0,1252.0,0.553967,0.322500,175.243093,8.886379
19035,False,2.392119e+06,41.125,-3.375,7.587690,3909.0,70.0,913.0,120.0,-15.901865,...,2779.0,1883.0,5335.0,3607.0,3526.0,1900.0,0.550767,0.299633,428.756907,2.329364
19036,False,2.035554e+06,31.125,12.250,10.487911,2361.0,133.0,91.0,200.0,-16.303966,...,918.0,712.0,1572.0,1011.0,1150.0,811.0,0.132200,0.099133,1976.756907,0.570856


In [125]:
data_high

,label,solar_radiance,terrain_roughness,terrain_prominence,slope,elevation,average_temperature,annual_precipitation,land_use_class,lat,...,ndvi-2019-09,evi-2019-09,ndvi-2020-03,evi-2020-03,ndvi-2020-09,evi-2020-09,summer_ndvi,winter_ndvi,elevation_from_mean,slope_from_mean
0,True,-0.599688,-0.303867,-0.487564,-0.598335,-1.193468,0.684233,-0.629381,130.0,-15.505033,...,2241.0,1481.0,6044.0,3914.0,2331.0,1596.0,1.823198,0.386372,0.698384,-0.304815
1,True,-0.226052,0.124071,-0.565500,-0.696705,-1.233266,0.892882,-1.076770,130.0,-15.530761,...,2715.0,1871.0,5646.0,3446.0,2579.0,1760.0,2.063924,0.962883,0.761793,-0.155786
2,True,-1.005219,0.804382,1.201044,-0.401643,-1.564923,1.205857,-1.072509,130.0,-15.607778,...,2064.0,1206.0,4141.0,2015.0,2254.0,1265.0,0.692889,0.163338,1.290204,-0.602802
3,True,0.722960,-0.583672,0.447665,-0.377443,0.488694,-0.724152,-1.238683,150.0,-16.910971,...,1153.0,754.0,1557.0,843.0,1248.0,624.0,-1.190508,-1.007929,-0.500112,-0.639464
4,True,0.690988,0.815354,0.772398,0.308062,0.491348,-0.463340,-0.676250,150.0,-17.040634,...,1480.0,936.0,2287.0,1024.0,1642.0,1024.0,-0.780348,-0.510779,-0.495884,-0.602563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19031,False,0.274123,0.864732,1.915456,0.010522,1.802054,-1.558750,2.302083,150.0,-15.351145,...,1184.0,683.0,54.0,56.0,1012.0,419.0,-1.528699,-0.956846,1.592397,-1.053332
19032,False,-0.894320,1.402397,2.071327,0.836548,0.637276,-0.776314,-0.003036,150.0,-15.243690,...,1736.0,1014.0,2430.0,1124.0,1983.0,968.0,-0.387560,-0.195157,-0.263383,0.198089
19033,False,0.590192,-1.242037,0.239836,-1.089950,0.515227,-0.619827,-1.149205,120.0,-16.335651,...,281.0,170.0,386.0,219.0,362.0,210.0,-1.949011,-2.212948,-0.457839,0.439977
19034,False,0.706495,-1.154255,-0.110875,-1.174877,0.488694,-0.463340,0.798004,130.0,-15.507890,...,2534.0,1596.0,5354.0,3411.0,4637.0,1252.0,1.544343,1.792987,-0.500112,0.568640


In [126]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [127]:
features = ["solar_radiance","terrain_roughness","terrain_prominence","slope",
            "slope_from_mean","elevation","elevation_from_mean", 
            "average_temperature","annual_precipitation","winter_ndvi","summer_ndvi"]#"land_use_class"]
scaler = StandardScaler()
data_high[features] = scaler.fit_transform(data_high[features])

In [129]:
for f in features:
    print(f)
    print(np.corrcoef(data[f],data['label'])[0,1])

solar_radiance
0.2392002583718008
terrain_roughness
-0.10014616086628716
terrain_prominence
-0.02653248295639015
slope
-0.10851907282983074
slope_from_mean
-0.2530083424332825
elevation
0.21867117137151443
elevation_from_mean
-0.30633766228868375
average_temperature
-0.21387807633348482
annual_precipitation
0.17644875708082863
winter_ndvi
0.1508994527455114
summer_ndvi
0.14436080224318212


In [140]:
features = ["solar_radiance","terrain_roughness","terrain_prominence","slope",
            "slope_from_mean","elevation","elevation_from_mean", 
            "average_temperature","annual_precipitation","winter_ndvi","summer_ndvi"]#"land_use_class"]
logreg_features = ["solar_radiance","terrain_roughness","slope",
            "slope_from_mean","elevation","elevation_from_mean", 
            "average_temperature","annual_precipitation","winter_ndvi","summer_ndvi"]#"land_use_class"]
X = data_high[features]
y = data_high['label']
logreg = LogisticRegression()
scores = cross_val_score(logreg, X, y, cv=20)
print('Mean accuracy:', scores.mean())
print('Standard deviation:', scores.std())

Mean accuracy: 0.659595139913492
Standard deviation: 0.027337130414256335


In [141]:
data_high[features].corr()

,solar_radiance,terrain_roughness,terrain_prominence,slope,slope_from_mean,elevation,elevation_from_mean,average_temperature,annual_precipitation,winter_ndvi,summer_ndvi
solar_radiance,1.000000,-0.546971,0.082408,-0.625399,-0.573186,0.617078,-0.331951,-0.540384,0.251103,-0.144331,-0.180764
terrain_roughness,-0.546971,1.000000,-0.020999,0.730839,0.397177,-0.124078,0.178886,0.070454,-0.094924,0.020204,-0.019117
terrain_prominence,0.082408,-0.020999,1.000000,-0.026236,-0.003346,0.029047,0.002023,0.015224,-0.012091,-0.015568,-0.017026
slope,-0.625399,0.730839,-0.026236,1.000000,0.587434,-0.077241,0.140735,0.039839,-0.055880,0.009812,-0.030552
slope_from_mean,-0.573186,0.397177,-0.003346,0.587434,1.000000,-0.166632,0.180601,0.138926,-0.123437,-0.000198,0.019758
elevation,0.617078,-0.124078,0.029047,-0.077241,-0.166632,1.000000,-0.372785,-0.934296,0.355349,-0.268394,-0.360594
elevation_from_mean,-0.331951,0.178886,0.002023,0.140735,0.180601,-0.372785,1.000000,0.284054,-0.219188,-0.042492,-0.004612
average_temperature,-0.540384,0.070454,0.015224,0.039839,0.138926,-0.934296,0.284054,1.000000,-0.132524,0.324844,0.424342
annual_precipitation,0.251103,-0.094924,-0.012091,-0.055880,-0.123437,0.355349,-0.219188,-0.132524,1.000000,0.243109,0.306571
winter_ndvi,-0.144331,0.020204,-0.015568,0.009812,-0.000198,-0.268394,-0.042492,0.324844,0.243109,1.000000,0.863492


In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
logreg.fit(X_train,y_train)
[m for m in zip(logreg_features,logreg.coef_[0])]

[('solar_radiance', 0.24162512262754057),
 ('terrain_roughness', 0.033213837221462206),
 ('slope', -0.024153126998151888),
 ('slope_from_mean', 0.3372218446565497),
 ('elevation', -0.5892664331600705),
 ('elevation_from_mean', -0.8774888502491931),
 ('average_temperature', -0.5594293757634946),
 ('annual_precipitation', -0.7756611080856313),
 ('winter_ndvi', 0.12051225053909159),
 ('summer_ndvi', 0.10325883305158273)]

In [143]:
print("Training set score: {:.3f}".format(logreg.score(X_train,y_train)))
print("Test set score: {:.3f}".format(logreg.score(X_test,y_test)))

import statsmodels.api as sm

logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary())

Training set score: 0.673
Test set score: 0.671
Optimization terminated successfully.
         Current function value: 0.617740
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                17817
Model:                          Logit   Df Residuals:                    17806
Method:                           MLE   Df Model:                           10
Date:                Thu, 20 Apr 2023   Pseudo R-squ.:                  0.1065
Time:                        20:36:54   Log-Likelihood:                -11006.
converged:                       True   LL-Null:                       -12319.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
solar_radiance           0.2798      0.0

In [139]:
logreg.coef_

array([[ 0.30529241,  0.10438786, -0.05654473,  0.30365962, -0.5476855 ,
        -0.91099157, -0.55596051, -0.77854777,  0.13034337,  0.06811896,
         0.23163588]])